In [2]:
from numpy import *

In [2]:
def loadDataset():
    """加载本地数据"""
    
    dataset = []
    f = open("./data/regData.txt")
    fr = f.readlines()
    for line in fr:
        line = line.strip().split("\t")
        linef = [float(li) for li in line]
        dataset.append(linef)
#         print(dataset)
    dataSetMat = mat(dataset)
    
    return dataSetMat

def calcErr(dataSetMat):
    
    error = var(dataSetMat[:,-1])*shape(dataSetMat)[0]
    
    return error

def chooseBestFeatureval2split(dataSetMat):
    """选择分类节点"""
    
    # 如果预测值相同
    if(len(set(dataSetMat[:, -1].T.tolist()[0])) == 1): return None, None
    
    bestFeature = 0
    bestValue = 0
    lowestErr = 100000
    totalErr = calcErr(dataSetMat)
    
    #  遍历特征
    for feature in range(shape(dataSetMat)[1] - 1):
        allValues = [d[feature] for d in dataSetMat.tolist()]
        values = set(allValues)  #每一个维度的特征
        for value in values:
            leftChild, rightChild = splitByFeatVal(feature, value, dataSetMat)
            if(shape(leftChild)[0] == 0 or shape(rightChild)[0] == 0): continue
            curErr = calcErr(leftChild) + calcErr(rightChild)
            if (curErr < lowestErr):
                bestFeature = feature
                bestValue = value
                lowestErr = curErr
        
    if (totalErr - lowestErr < 1): return None, None # 防止过拟合，并且作为递归的终止条件
    
    return bestFeature, bestValue

def splitByFeatVal(feature, value, dataSetMat):
    """分裂节点"""
    
    leftChild = dataSetMat[nonzero(dataSetMat[:, feature] > value)[0],:]
    rightChild = dataSetMat[nonzero(dataSetMat[:, feature] <= value)[0],:]
    
    return leftChild, rightChild

def createRegTree(dataSetMat):
    """创建回归树"""
    
    feature, value = chooseBestFeatureval2split(dataSetMat)
    
    if feature == None: return mean(dataSetMat[:, -1])#返回最终的预测结果
    
    regTree = {}
    regTree["feature"] = feature
    regTree["value"] = value
    leftChild, rightChild = splitByFeatVal(feature, value, dataSetMat)
    regTree["leftChild"] = createRegTree(leftChild)
    regTree["rightChild"] = createRegTree(rightChild)
    
    return regTree

if __name__ == "__main__":
    dataSetMat = loadDataset()
    regTree = createRegTree(dataSetMat)
    print(regTree)

{'feature': 1, 'value': 10.0, 'leftChild': {'feature': 1, 'value': 20.0, 'leftChild': 20.268, 'rightChild': {'feature': 1, 'value': 15.0, 'leftChild': 15.266, 'rightChild': 10.27}}, 'rightChild': {'feature': 1, 'value': 5.0, 'leftChild': 5.07, 'rightChild': 0.078}}
